In [1]:
# Check GPU configuration
!nvidia-smi

Sun Dec 31 04:37:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title STEP 0 連接 Google Drive
#@markdown #STEP 0
#@markdown ##連接 Google Drive
#@markdown ##Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#STEP 0 確認有使用GPU
##記事本必須在GPU模式下運行，可以執行上方的nvidia-smi查看GPU是否有GPU。若没有，點擊左上角的 編輯>筆記本設定，把硬體加速器改成GPU。
##如記事本有出問題可在以下儲存庫回報
> https://github.com/ADT109119/Bert-VITS2-Colab

In [ ]:
#@title STEP 1 複製儲存庫並安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##複製儲存庫並安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib
#@markdown ###
#@markdown ###選擇訓練版本
training_mode = "一般版本" # @param ["一般版本", "中文特化版"]
training_modes = ["一般版本", "中文特化版"]
if training_mode == training_modes[0]:
    !git clone https://github.com/fishaudio/Bert-VITS2.git
elif training_mode == training_modes[1]:
    !git clone https://github.com/fishaudio/Bert-VITS2.git --branch Extra-Fix
%cd ./Bert-VITS2/
!pip install -r requirements.txt
!pip install pyyaml

#下載 Bert 以及 wavlm 模型
!wget https://huggingface.co/microsoft/wavlm-base-plus/resolve/main/pytorch_model.bin?download=true -O slm/wavlm-base-plus/pytorch_model.bin

if training_mode == training_modes[0]:
    !rm -rf bert/chinese-roberta-wwm-ext-large
    !git clone https://huggingface.co/hfl/chinese-roberta-wwm-ext-large bert/chinese-roberta-wwm-ext-large
    !rm -rf bert/deberta-v2-large-japanese-char-wwm
    !git clone https://huggingface.co/ku-nlp/deberta-v2-large-japanese-char-wwm bert/deberta-v2-large-japanese-char-wwm
    !rm -rf bert/deberta-v3-large
    !git clone https://huggingface.co/microsoft/deberta-v3-large bert/deberta-v3-large

elif training_mode == training_modes[1]:
    !rm -rf bert/Erlangshen-MegatronBert-1.3B-Chinese
    !git clone https://huggingface.co/IDEA-CCNL/Erlangshen-MegatronBert-1.3B bert/Erlangshen-MegatronBert-1.3B-Chinese
    !rm -rf emotional/clap-htsat-fused
    !git clone https://huggingface.co/laion/clap-htsat-fused emotional/clap-htsat-fused
    !rm -rf emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim
    !git clone https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim
    !wget https://huggingface.co/ADT109119/G2PWModel-v2-onnx/resolve/main/g2pw.onnx?download=true -O g2pW/g2pW.onnx
  

# Bert-VITS2 数据预处理

数据准备：
将数据放置在 data 文件夹下，按照如下结构组织：

```
├── data
│   ├── {你的数据集名称}
│   │   ├── esd.list
│   │   ├── raw
│   │   │   ├── ****.wav
│   │   │   ├── ****.wav
│   │   │   ├── ...
```

其中，`raw` 文件夹下保存所有的音频文件，`esd.list` 文件为标签文本，格式为

```
****.wav|{说话人名}|{语言 ID}|{标签文本}\n
```

例如：
```
vo_ABDLQ001_1_paimon_02.wav|派蒙|ZH|没什么没什么，只是平时他总是站在这里，有点奇怪而已
noa_501_0001.wav|NOA|JP|そうだね、油断しないのはとても大事なことだと思う\n"
Albedo_vo_ABDLQ002_4_albedo_01.wav|Albedo|EN|Who are you? Why did you alarm them?
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title STEP 2-1 生成上傳按鈕載入資料集
#@markdown #STEP 2-1
#@markdown ##生成上傳按鈕載入資料集
#@markdown ##Upload Dataset

from google.colab import files
import shutil
import os
uploaded = files.upload()
basepath = os.getcwd()
upload_path = "./"
for filename in uploaded.keys():
  shutil.move(os.path.join(basepath, filename), os.path.join(upload_path, "data.zip"))


In [ ]:
#@title STEP 2-2 從Google Drive載入資料集
#@markdown #STEP 2-2
#@markdown ##載入資料集
#@markdown ##Upload Dataset from Google Drive

#@markdown ###資料集路徑
data_set_zip_file = "/content/drive/MyDrive/Bert-VITS2/data.zip" #@param {type:"string"}

!cp -rf {data_set_zip_file} ./data.zip



In [ ]:
#@title STEP 2-3 上傳音檔 透過Whisper生成字幕再自動切分 (因準確度關係 不建議使用)
#@markdown #STEP 2-3
#@markdown ##透過Whisper生成字幕再自動切分
#@markdown ##(因準確度關係 不建議使用)
#@markdown ##Upload audios and generate subtitle by OpenAI Whisper to split audios
#@markdown ##(Not recomand)
#@markdown ##音檔請依照 `{說話者名稱}_{語言}_{隨便的編號}` 格式

!apt install ffmpeg

!git clone https://github.com/ADT109119/Bert-VITS2-colab.git upload_audios

!mkdir upload_audios
!mkdir upload_audios/audio
!mkdir upload_audios/srt

upload_from_google_drive = True # @param {type:"boolean"}

if upload_from_google_drive:
  data_folder_path = "/content/drive/MyDrive/Bert-VITS2/audio" #@param {type:"string"}
  !cp -rf {data_folder_path}/* ./upload_audios/audio
else:
  from google.colab import files
  import shutil
  import os
  uploaded = files.upload()
  basepath = os.getcwd()
  upload_path = "./upload_audios/audio"
  for filename in uploaded.keys():
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_path, filename))

!pip install openai-whisper
import glob

file_list = glob.glob("upload_audios/audio/*.wav")
for path in file_list:
  !whisper {path} --model large-v3 --output_dir "./upload_audios/srt"

!python ./upload_audios/processing.py
import zipfile
import os

def zip_folder_with_file(folder_path, file_path, output_path):
    with zipfile.ZipFile(output_path, 'w') as zip_file:
        for foldername, subfolders, filenames in os.walk(folder_path):
            for filename in filenames:
                file_path_in_folder = os.path.join(foldername, filename)
                zip_file.write(file_path_in_folder, os.path.relpath(file_path_in_folder, os.path.dirname(folder_path)))

        zip_file.write(file_path, os.path.basename(file_path))

zip_folder_with_file('./upload_audios/raw', './upload_audios/esd.list', './data.zip')


In [ ]:
#@title STEP 2.1 解壓縮資料集
#@markdown #STEP 2.1
#@markdown ##解壓縮資料集
#@markdown ##Unzip Data

#@markdown ###資料集名稱
!mkdir data

data_dir = "dataset" #@param {type:"string"}
!unzip ./data.zip -d ./data/{data_dir}

In [ ]:
#@title STEP 2.5 處理資料
#@markdown #STEP 2.5
#@markdown ##處理資料
#@markdown ##Process Data
from webui_preprocess import generate_config, resample, preprocess_text, bert_gen
batch_size = 8 # @param {type:"integer"}
print(generate_config(data_dir, batch_size))
print(resample(data_dir))
print(preprocess_text(data_dir))

#修改 config.yml
import yaml
def load_yaml(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data
def save_yaml(data, file_path):
    with open(file_path, 'w', encoding="utf-8") as file:
        yaml.dump(data, file, default_flow_style=False)
# 載入 YAML 檔案
yaml_file_path = 'config.yml'
yaml_data = load_yaml(yaml_file_path)
# 修改 YAML 中的內容
yaml_data['dataset_path'] = f"data/{data_dir}"
yaml_data['train_ms']['config_path'] = "configs/config.json"
if training_mode == training_modes[1]:
    yaml_data['bert_gen']['num_processes'] = 1

# 儲存修改後的 YAML 檔案
save_yaml(yaml_data, yaml_file_path)
print(bert_gen(data_dir))

if training_mode == training_modes[1]:
    !python clap_gen.py --config data/{data_dir}/configs/config.json

In [ ]:
#@title STEP 2.6 自動儲存步數修改
#@markdown #STEP 2.6
#@markdown ##自動儲存步數修改

#@markdown ###每隔n步自動儲存
save_step = 200 # @param {type:"integer"}

import json

def load_json(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        data = json.load(file)
    return data

def save_json(data, file_path):
    with open(file_path, 'w', encoding="utf-8") as file:
        json.dump(data, file, indent=2)

# 載入 JSON 檔案
json_file_path = f"data/{data_dir}/configs/config.json"
json_data = load_json(json_file_path)

# 修改 JSON 中的內容
json_data['train']['log_interval'] = save_step
json_data['train']['eval_interval'] = save_step

# 儲存修改後的 JSON 檔案
save_json(json_data, json_file_path)


# STEP 3 載入模型
## 3-1、3-2 則一

In [ ]:
#@title STEP 3-1 下載預訓練模型
if training_mode == training_modes[0]:
    !git clone https://huggingface.co/OedoSoldier/Bert-VITS2-2.3 data/{data_dir}/models
elif training_mode == training_modes[1]:
    !git clone https://huggingface.co/v3ucn/Bert-vits2-Extra-Pretrained_models data/{data_dir}/models

In [ ]:
#@title STEP 3-2 載入上次訓練到一半 儲存在 Google Drive 的模型
!cp -f ../drive/MyDrive/Bert-VITS2/models/* data/{data_dir}/models

#STEP 4 開始訓練

In [ ]:
import os
os.environ['TENSORBOARD_BINARY'] = '/usr/local/bin/tensorboard'
%reload_ext tensorboard
model_dir = f"./data/{data_dir}/models"
%tensorboard --logdir $model_dir

!torchrun --nproc_per_node=1 train_ms.py

#STEP 5 推理測試


In [ ]:
#@title STEP 5 推理測試
#@markdown #STEP 5
#@markdown ##推理測試

#@markdown ###
#@markdown ###推理所使用的模型步數
model_step = 2000 # @param {type:"integer"}
import yaml
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data
def save_yaml(data, file_path):
    with open(file_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)
# 載入 YAML 檔案
yaml_file_path = 'config.yml'
yaml_data = load_yaml(yaml_file_path)
# 修改 YAML 中的內容
yaml_data['dataset_path'] = f"data/{data_dir}"
yaml_data['webui']['share'] = "true"
yaml_data['webui']['model'] = f"models/G_{model_step}.pth"
yaml_data['webui']['config_path'] = "configs/config.json"
# 儲存修改後的 YAML 檔案
save_yaml(yaml_data, yaml_file_path)
!torchrun --nproc_per_node=1 webui.py

#STEP 6 保存到Google Drive(自選)

In [ ]:
#@title STEP 6.1 連接Google Drive
#@markdown ##如果前面連接過了 這邊就不用再連一次
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 6.2 複製檔案到Google Drive
#@markdown ###模型儲存路徑
Google_Drive_Folder = "/content/drive/MyDrive/Bert-VITS2" #@param {type:"string"}
#@markdown ###儲存模型步數
steps = 2000 # @param {type:"integer"}
!mkdir {Google_Drive_Folder}/models
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/G_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/D_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/WD_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/models/DUR_{steps}.pth {Google_Drive_Folder}/models/
!cp -rf /content/Bert-VITS2/data/{data_dir}/configs {Google_Drive_Folder}


#STEP 7 下載模型(自選)

In [ ]:
#@markdown ###下載模型步數
from google.colab import files
steps = 2000 # @param {type:"integer"}
files.download(f"./data/{data_dir}/models/G_{steps}.pth")
files.download(f"./data/{data_dir}/models/D_{steps}.pth")
files.download(f"./data/{data_dir}/models/WD_{steps}.pth")
files.download(f"./data/{data_dir}/models/DUR_{steps}.pth")
files.download(f"./data/{data_dir}/configs/config.json")

#STEP 8 一鍵部署到 HuggingFace 上 (尚不支持中文特化版)

In [ ]:
#@title STEP 8.1 使用API Token登入
#@markdown #STEP 8.1
#@markdown ##執行本儲存格後填入 Access Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the Access Token (Write permission is required), and then click Login
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#@title STEP 8.2 上傳模型
#@markdown #STEP 8.2
#@markdown ##在此儲存格填入 `HF 儲存庫 ID`、`欲使用的模型 Step`、`作者`......等資訊
#@markdown ##執行後便可創建 Space
from huggingface_hub import HfApi
api = HfApi()

#複製模板
!rm -rf Bert-VITS2-Docker-template
if training_mode == training_modes[0]:
    !git clone https://github.com/ADT109119/Bert-VITS2-Docker-template.git
elif training_mode == training_modes[1]:
    !git clone https://github.com/ADT109119/Bert-VITS2-Docker-template.git --branch Chinese

#@markdown ###
#@markdown ###使用的模型步數:
model_step = 2000 # @param {type:"integer"}
import yaml
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data
def save_yaml(data, file_path):
    with open(file_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

yaml_data = load_yaml("Bert-VITS2-Docker-template/config.yml")
yaml_data['dataset_path'] = f"Data/{data_dir}"
yaml_data['webui']['model'] = f"models/G_{model_step}.pth"
save_yaml(yaml_data, "Bert-VITS2-Docker-template/config.yml")

#@markdown ###儲存庫ID:
repo_id_ = "username/Bert-VITS2" #@param {type:"string"}
#@markdown ###Space 名稱:
space_name = "Bert-VITS2" #@param {type:"string"}
#@markdown ###作者:
space_author = "作者" #@param {type:"string"}
#@markdown ###聲音歸屬:
voice_ = "聲音屬於 xxx" #@param {type:"string"}
#@markdown ###是否為私人Space (打勾為不公開):
private_ = True # @param {type:"boolean"}

import json
def load_json(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        data = json.load(file)
    return data

def save_json(data, file_path):
    with open(file_path, 'w', encoding="utf-8") as file:
        json.dump(data, file, indent=2)

json_data = load_json("Bert-VITS2-Docker-template/author_and_voice_data.json")
json_data['author'] = space_author
json_data['voice'] = voice_
save_json(json_data, "Bert-VITS2-Docker-template/author_and_voice_data.json")

#複製模型與Config
!mkdir Bert-VITS2-Docker-template/Data/{data_dir}
!mkdir Bert-VITS2-Docker-template/Data/{data_dir}/models
!cp data/{data_dir}/models/G_{model_step}.pth Bert-VITS2-Docker-template/Data/{data_dir}/models
!cp data/{data_dir}/configs/config.json Bert-VITS2-Docker-template/Data/{data_dir}

!rm -rf Bert-VITS2-Docker-template/.git

with open('Bert-VITS2-Docker-template/README.md', 'r+', encoding='utf-8') as fr:
  content = fr.read()
  content = content.replace("title: Bert VITS2 Docker Template", f"title: {space_name}")
  fr.seek(0)
  fr.write(content)
  fr.truncate()

api.create_repo(repo_id_, private=private_, repo_type="space", space_sdk="docker")
api.upload_folder(
    repo_type="space",
    folder_path='Bert-VITS2-Docker-template/',
    repo_id=repo_id_,
)